In [8]:
# Importing required packages and dependencies
from dataclasses import dataclass
from typing import List, Dict, Optional
import openai
import json
from openai import OpenAI
import re

In [9]:
#setting up openai key
api = "<enter your key>"

client = OpenAI(api_key=api)

In [10]:
#Helper function for formatting
def strip_markdown_json(content: str) -> str:
    # Matches ```json\n{...}\n``` or just ```\n{...}\n```
    pattern = r"```(?:json)?\s*([\s\S]*?)\s*```"
    match = re.search(pattern, content)
    return match.group(1).strip() if match else content.strip()



In [11]:

@dataclass
class StudentProfile:
    name: str
    content_access: List[str]
    subject_stats: Dict[str, Dict[str, float]]
    knowledge_graph: Dict[str, Dict[str, Dict[str, Optional[str]]]]
    session_history: List[Dict[str, str]]
    preference: Dict[str, any]

@dataclass
class TutoringMessage:
    role: str
    message_type: str
    payload: Dict

class ReceptionistAgent:
    def __init__(self, profile: StudentProfile):
        self.profile = profile

    def handle_request(self, query: str) -> TutoringMessage:
      system_prompt = """
      You are the receptionist agent for an intelligent tutoring system.

      Your job is to:
      1. Interpret the student's intent from their natural language query. Possible intents include:
          - "study_request": the student wants to learn new material
          - "review_request": the student wants to revise known material
          - "help_request": the student is struggling and needs help
      2. Choose the subject most relevant to their query based on subject_stats (progress, mastery).
      3. Return ONLY a JSON object with the following keys:
          - "intent": (one of the intent labels above)
          - "target_subject": (e.g., "Algebra", "Geometry")
          - "context": an object containing "progress" and "mastery" for that subject
      """

      user_prompt = f"""
      Student query: "{query}"

      Student subject stats:
      {json.dumps(self.profile.subject_stats, indent=2)}

      Return your result as **only** a JSON object like this:
      {{
        "intent": "...",
        "target_subject": "...",
        "context": {{
          "progress": ...,
          "mastery": ...
        }}
      }}
      """

      response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
              {"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}
          ]
      )

      content = strip_markdown_json(response.choices[0].message.content)
      return TutoringMessage("Receptionist", "intent", json.loads(content))


class SubjectPlannerAgent:
    def __init__(self, profile: StudentProfile):
        self.profile = profile

    def propose_plan(self, subject: str, context: Dict, intent: str,
                     feedback: Optional[str] = None, original_plan: Optional[Dict] = None) -> TutoringMessage:
        if feedback and original_plan:
            system_prompt = """
            You are the subject planner agent. The student has feedback on the proposed plan.
            Revise it while preserving pedagogical intent and flow.
            Do not remove core activities without offering substitutes.
            All selected topics and modules must belong to the student's content_access list.
            """

            user_prompt = f"""
            Original plan: {json.dumps(original_plan, indent=2)}
            Feedback: {feedback}
            Student intent: {intent}
            Content access: {json.dumps(self.profile.content_access, indent=2)}

            Return revised plan ONLY as JSON with:
            learning_objective, pedagogy, and activity_sequence.
            """
            message_type = "revised_plan"
        else:
            system_prompt = """
            You are the subject planner agent. Propose a personalized learning plan with pedagogical integrity.

            Use the student's:
            - intent (e.g., study_request, help_request, review_request)
            - mastery level
            - session history
            - knowledge graph
            - preferences
            - content_access list

            You must only select topics or modules that exist in the content_access list.
            Return ONLY a valid JSON object with the required fields.
            """

            user_prompt = f"""
            Subject: {subject}
            Intent: {intent}
            Mastery: {context['mastery']}
            Session history: {json.dumps(self.profile.session_history, indent=2)}
            Knowledge graph: {json.dumps(self.profile.knowledge_graph.get(subject, {}), indent=2)}
            Student preferences: {json.dumps(self.profile.preference, indent=2)}
            Content access: {json.dumps(self.profile.content_access, indent=2)}

            Return JSON with:
            - learning_objective
            - pedagogy
            - target_subject
            - activity_sequence (array of learning tasks with topic, activity type, and constraints)
            """
            message_type = "plan"

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        content = strip_markdown_json(response.choices[0].message.content)
        return TutoringMessage("Planner", message_type, json.loads(content))



In [13]:
#running the function

def main():
    profile = StudentProfile(
    name="Alice",
    content_access=[
        "Algebra_Basics", "Algebra_Quadratics", "Algebra_Polynomials",
        "Geometry_Basics", "Geometry_Triangles", "Geometry_Circles",
        "Calculus_Limits", "Calculus_Derivatives"
    ],
    subject_stats={
        "Algebra": {"progress": 75, "mastery": 0.68},
        "Geometry": {"progress": 85, "mastery": 0.80},
        "Calculus": {"progress": 40, "mastery": 0.55}
    },
    knowledge_graph={
        "Algebra": {
            "linear_equations": {"status": "mastered", "last_seen": "2025-04-10"},
            "quadratic_equations": {"status": "partial", "last_seen": "2025-04-25"},
            "polynomials": {"status": "unseen", "last_seen": None}
        },
        "Geometry": {
            "angles": {"status": "mastered", "last_seen": "2025-04-15"},
            "triangles": {"status": "partial", "last_seen": "2025-04-22"},
            "circles": {"status": "unseen", "last_seen": None}
        },
        "Calculus": {
            "limits": {"status": "partial", "last_seen": "2025-04-28"},
            "derivatives": {"status": "unseen", "last_seen": None},
            "integrals": {"status": "unseen", "last_seen": None}
        }
    },
    session_history=[
        {"date": "2025-04-28", "activity": "quiz", "topic": "limits", "score": "0.5"},
        {"date": "2025-04-27", "activity": "interactive_simulation", "topic": "limits"},
        {"date": "2025-04-26", "activity": "video", "topic": "triangles", "watched": "True"},
        {"date": "2025-04-25", "activity": "quiz", "topic": "quadratic_equations", "score": "0.6"},
        {"date": "2025-04-24", "activity": "video", "topic": "quadratic_equations", "watched": "True"},
        {"date": "2025-04-23", "activity": "group_discussion", "topic": "angles"},
        {"date": "2025-04-20", "activity": "quiz", "topic": "linear_equations", "score": "0.85"}
    ],
    preference={
        "video": False,
        "interactive": True,
        "quiz_difficulty": "medium",
        "time_limit": 10,
        "preferred_subject": "Algebra",
        "max_session_duration": 30,  # minutes
        "preferred_learning_style": "inquiry-based",
        "avoid_activities": ["lecture_notes", "long videos"]
    }
)


    receptionist = ReceptionistAgent(profile)
    planner = SubjectPlannerAgent(profile)

    query = input("What would you like to study today (Algebra, Geometry, Calculus)? > ").strip()
    intent_msg = receptionist.handle_request(query)
    print("\n Receptionist Output:")
    print(json.dumps(intent_msg.payload, indent=2))


    plan_msg = planner.propose_plan(
        subject=intent_msg.payload["target_subject"],
        context=intent_msg.payload["context"],
        intent=intent_msg.payload["intent"]
    )
    print("\n Tutoring Plan:")
    print(json.dumps(plan_msg.payload, indent=2))

    feedback = input("\n Any feedback on this plan? (Leave blank to accept) > ").strip()
    if feedback:
        revised_msg = planner.propose_plan(
            subject=intent_msg.payload["target_subject"],
            context=intent_msg.payload["context"],
            intent=intent_msg.payload["intent"],
            feedback=feedback,
            original_plan=plan_msg.payload
        )
        print("\n Updated Plan after Negotiation:")
        print(json.dumps(revised_msg.payload, indent=2))
    else:
        print("\n Plan accepted. You're ready to begin!")


if __name__ == "__main__":
    main()


What would you like to study today (Algebra, Geometry, Calculus)? > revise geometry

 Receptionist Output:
{
  "intent": "review_request",
  "target_subject": "Geometry",
  "context": {
    "progress": 85,
    "mastery": 0.8
  }
}

 Tutoring Plan:
{
  "learning_objective": "Review and reinforce understanding of triangles and familiarize with circles in geometry.",
  "pedagogy": "Inquiry-based learning through interactive exploration and problem-solving activities.",
  "target_subject": "Geometry",
  "activity_sequence": [
    {
      "topic": "Geometry_Triangles",
      "activity_type": "interactive_simulation",
      "constraints": {
        "time_limit": 10,
        "session_duration": 30
      }
    },
    {
      "topic": "Geometry_Circles",
      "activity_type": "interactive_simulation",
      "constraints": {
        "time_limit": 10,
        "session_duration": 30
      }
    },
    {
      "topic": "Geometry_Triangles",
      "activity_type": "quiz",
      "constraints": {
   